In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot
import pandas as pd
import numpy as np

In [3]:
# Definimos colores fundar
fundar_colores_primarios = ['#7BB5C4', '#9FC1AD', '#D3D3E0',  '#8d9bff', '#FF9750', '#FFD900',]
fundar_colores_secundarios = ['#B5E0EA', '#B3B3B3', '#848279', '#AFA36E', '#5D896F', '#9C9CBC', '#E27124']
fundar_white = '#F5F5F5'
fundar_black = '#151515'

In [4]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'


In [5]:
ratio_actividad = pd.read_csv(pathdata + 'data_output/indicadores/01_ratio_actividad.csv', index_col=0)
empleo_asalariado_con_descuento = pd.read_csv(pathdata + 'data_output/indicadores/02_empleo_asalariado_con_descuento.csv', index_col=0)
horas_remuneradas_media = pd.read_csv(pathdata + 'data_output/indicadores/03_horas_promedio_ocup_ppal.csv', index_col=0)
ingreso_salarial_media = pd.read_csv(pathdata + 'data_output/indicadores/04_ingreso_salarial.csv', index_col=0)
ratio_minutos_promedio_no_pago = pd.read_csv(pathdata + 'data_output/indicadores/05_tiempo_no_remunerado.csv', index_col=0)
inactivos_con_ingreso = pd.read_csv(pathdata + 'data_output/indicadores/06_dependencia_economica.csv', index_col=0)
hogares_jefatura = pd.read_csv(pathdata + 'data_output/indicadores/07_hogares_no-pobres_jefatura.csv', index_col=0)

In [6]:
variables = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas_media, ingreso_salarial_media, ratio_minutos_promedio_no_pago, inactivos_con_ingreso, hogares_jefatura]
columnas = ['Actividad', 'Formalidad', 'Horas promedio remuneradas', 'Ingreso laboral promedio', 'Tiempo TNR', 'Dependencia', 'No Pobreza']

for columna, variable in zip(columnas,variables):
    variable[columna] = np.where(variable[columna] > 100, 100, variable[columna])


In [7]:
componente_insercion = pd.read_csv(pathdata + 'data_output/componentes_e_indice/01_insercion_laboral.csv', index_col=0)
componente_autonomia_economica = pd.read_csv(pathdata + 'data_output/componentes_e_indice/02_insercion_autonomia_economica.csv', index_col=0)
indice_mti = pd.read_csv(pathdata + 'data_output/componentes_e_indice/01_indice_MTeI.csv', index_col=0)

In [12]:
import plotly.graph_objects as go
from plotly.offline import plot

bar_trace1 = go.Bar(
    x=ratio_actividad.index,
    y=ratio_actividad['Actividad'],
    name='Actividad',
    marker_color=fundar_colores_primarios[0]
)

bar_trace2 = go.Bar(
    x=empleo_asalariado_con_descuento.index,
    y=empleo_asalariado_con_descuento['Formalidad'],
    name='Formalidad',
    marker_color=fundar_colores_primarios[1]
)

bar_trace3 = go.Bar(
    x=horas_remuneradas_media.index,
    y=horas_remuneradas_media['Horas promedio remuneradas'],
    name='Horas promedio remuneradas',
    marker_color=fundar_colores_primarios[2]
)

bar_trace4 = go.Bar(
    x=ingreso_salarial_media.index,
    y=ingreso_salarial_media['Ingreso laboral promedio'],
    name='Ingreso laboral promedio',
    marker_color=fundar_colores_primarios[3]
)

bar_trace5 = go.Bar(
    x=componente_insercion.index,
    y=componente_insercion['Insercion laboral'],
    name='Componente',
    marker_color=fundar_colores_primarios[4]
)

layout = go.Layout(
    title='Indicadores e insercion laboral',
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    font_size=14
)

fig = go.Figure(data=[bar_trace1, bar_trace2, bar_trace3, bar_trace4, bar_trace5], layout=layout)

fig.add_shape(
    type='line',
    x0=ratio_actividad.index[0],
    y0=100,
    x1=ratio_actividad.index[-1],
    y1=100,
    line=dict(
        color='black',
        width=1,
        dash='dash'
    ),
)

file_path = pathdata + 'figs/'
filename = 'insercion_laboral.html'
plot(fig, filename=file_path + filename, auto_open=False)

'/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/insercion_laboral.html'

In [17]:
bar_trace1 = go.Bar(
    x=ratio_minutos_promedio_no_pago.index,
    y=ratio_minutos_promedio_no_pago['Tiempo TNR'],
    name='Tiempo TNR',
    marker_color=fundar_colores_primarios[0]
)

bar_trace2 = go.Bar(
    x=inactivos_con_ingreso.index,
    y=inactivos_con_ingreso['Dependencia'],
    name='Dependencia',
    marker_color=fundar_colores_primarios[1]
)

bar_trace3 = go.Bar(
    x=hogares_jefatura.index,
    y=hogares_jefatura['No Pobreza'],
    name='No Pobreza',
    marker_color=fundar_colores_primarios[2]
)

bar_trace4 = go.Bar(
    x=componente_autonomia_economica.index,
    y=componente_autonomia_economica['Autonomia economica'],
    name='Autonomia economica',
    marker_color=fundar_colores_primarios[3]
)

layout = go.Layout(
    title='Indicadores y uso del tiempo y oportunidades',
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    font_size=14
)

fig = go.Figure(data=[bar_trace1, bar_trace2, bar_trace3, bar_trace4], layout=layout)

fig.add_shape(
    type='line',
    x0=ratio_minutos_promedio_no_pago.index[0],
    y0=100,
    x1=ratio_minutos_promedio_no_pago.index[-1],
    y1=100,
    line=dict(
        color='black',
        width=1,
        dash='dash'
    ),
)

file_path = pathdata + 'figs/'
filename = 'uso_del_tiempo_y_oportunidades.html'
plot(fig, filename=file_path + filename, auto_open=False)

'/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/uso_del_tiempo_y_oportunidades.html'